In [1]:
import sys
import os

# Add the parent directory to sys.path
parent_dir = os.path.abspath("../")
print(parent_dir)
sys.path.append(parent_dir)


/Users/hanna/openfn/ai_experiments/apollo/services


In [2]:
from search_docsite import DocsiteSearch
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import os
import json
from dotenv import load_dotenv
import anthropic
from embed_docsite import github_utils

load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
docsite_search = DocsiteSearch(collection_name="docsite-20250225")
results = docsite_search.search("what's asana", top_k=10)
results

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 10


[SearchResult(text='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out the Asana [devloper documentation](/adaptors/packages/asana-docs) to learn how to set up a webhook to push data

In [15]:
user_questions = [
    "How do I modify a contact on Salesforce"
    'Write a job for me that creates new DataValueSets under the "Approved School CHP" organization unit.',
    "How to get data from Asana in my job?",
    "What's the difference between a Trigger and a Step in a workflow?",
    "Can I write a step that allows for creation of a JSON file of the transformed records that can be sent to an email or Google Drive?",
    "Why does this part of the code only POST the first submission in the submissions[] array? I want to POST each submission. each( '$.submissions[*]', post(URL, { body: () => {} ))",
    "Write a job to fetch for an attribute in the input and map it.",
    "Can I set up a workflow to trigger based on changes in a database using a webhook? How?",  
    "How can I securely manage API keys and other sensitive information using environment variables in OpenFn?",  
    "Does OpenFn offer any built-in support for encrypting sensitive data in transit or at rest?",  
    "How can I customize the logging level and output format for my workflows?",  
    "What's the recommended workflow for using Git to manage and deploy changes to my OpenFn workflows?",  
    "How can I write unit tests for my OpenFn jobs to ensure they are working correctly?",  
    "How can I set up alerts to notify me when a workflow fails or exceeds a certain execution time?",
    "How can I use the post() function with the HTTP adapter to send transformed data to another system?",
    "What's the syntax for using the create() function with the DHIS2 adapter to create new data elements?",
    "How do I use the alterState() function to modify the state object between operations?"
]


In [ ]:
# testx threshold
# estimate a threshold to use in sim search

docsite_search = DocsiteSearch(collection_name="docsite-20250225")
results = docsite_search.search("hCan I write a step that allows for creation of an JSON file of the transformed records that can be sent to an email or Google Drive", top_k=20) #threshold=0.73
results

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 20
INFO:DocsiteSearch:Filtered to 20 results


[SearchResult(text='ng data from CRM systems into Google Sheets for sales\nanalysis. - Updating inventory levels in a Google Sheets spreadsheet from an\ne-commerce platform. - Automatically populating project management data from\ntask tracking systems into Google Sheets for reporting purposes.\n\nA step by step guide is found\n[in this tutorial](https://docs.openfn.org/documentation/tutorials/http-to-googlesheets)\nthat shows us how to get data via a REST API and push it to Google Sheet.### Helpful Resources\n\n- [Google Sheets API Overview](https://developers.google.com/sheets/api/guides/concepts)\n- OpenFn Workflow Tutorial:\n  [HTTP-to-GoogleSheets](https://docs.openfn.org/documentation/tutorials/http-to-googlesheets)', metadata={'doc_title': 'googlesheets', 'docs_type': 'adaptor_docs'}, score=0.79843092),
 SearchResult(text='- You have access to an OpenFn project (either on a locally installed\n  [OpenFn v2 app](https://github.com/OpenFn/lightning) or on\n  [app.openfn.org](https:

In [16]:
# estimate a threshold to use in sim search
def determine_threshold(user_questions):
    docsite_search = DocsiteSearch(collection_name="docsite-20250225")
    
    results_dict = {}
    for question in user_questions:
        results = docsite_search.search(question, top_k=30)
        results_dict[question] = results
    
    threshold_prompt = """
    You are an expert in determining relevant document retrieval thresholds.
    Given a user question, a retrieved search result, and its relevance score,
    determine a reasonable threshold score for filtering relevant results.
    
    The input data follows this format:
    - User Question: {user_question}
    - Search Result: {search_result}
    - Relevance Score: {score}
    
    Provide 2–3-sentence reasoning and a single numerical threshold value as output.
    """
    
    def llm_determine_threshold(user_question, results):
        prompt_content = "\n".join([
            threshold_prompt.format(
                user_question=user_question,
                search_result=r.text,
                score=r.score
            ) for r in results
        ])
        
        message = client.messages.create(
            model="claude-3-5-sonnet-20241022",  
            max_tokens=300,
            temperature=0,
            system="Decide an appropriate threshold for filtering relevant search results.",
            messages=[
                {"role": "user", "content": [{"type": "text", "text": prompt_content}]}
            ]
        )
        
        return message.content[0].text.strip()
    
    thresholds = {}
    for question, results in results_dict.items():
        thresholds[question] = llm_determine_threshold(question, results)
        
    return thresholds
    

In [17]:
result = determine_threshold(user_questions)
result

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 30 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 30 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 30 results

{'How do I modify a contact on SalesforceWrite a job for me that creates new DataValueSets under the "Approved School CHP" organization unit.': "Looking at the search results and their relevance scores, I notice that the scores cluster between 0.75-0.79, with the most relevant result having a score of 0.789996564. The first result contains directly relevant DHIS2 DataValueSet creation code, while results below 0.78 tend to be less directly relevant to the user's question about creating DataValueSets. Based on this pattern, I would recommend a threshold of 0.78 to filter for the most relevant results while excluding less relevant examples.\n\nThreshold value: 0.78",
 'How to get data from Asana in my job?': 'Based on the search results shown, I can see that the most relevant result has a score of 0.814727 and contains direct information about Asana integration options, while results below 0.80 start to become less relevant to the specific question about getting data from Asana. Looking 

In [26]:
df = pd.read_csv("results_v1_top3.csv")
df

,question,question_type,consult_doc_label,consult_adaptor_docs_label,consult_general_docs_label,consult_doc_answer_v1,consult_adaptor_docs_answer_v1,consult_general_docs_answer_v1,search_query_answer_v1,search_results_answer_v1,evaluation,evaluations_bin
0,Is there a way to handle pagination when pulli...,adaptor_questions,True,True,False,False,False,False,"[{'query': 'pagination memory large dataset', ...",[[SearchResult(text='This typically won\'t hap...,True\n\nThe second snippet discusses cursor-ba...,True
1,When authenticating with API tokens instead of...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'DHIS2', 'doc_type': 'adaptor_docs'...","[[SearchResult(text=""# DHIS2## App Overview\n\...",False\n\nThe retrieved documentation snippets ...,False
2,I need to update a Contact record in Salesforc...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'salesforce', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: Salesforce\n-...,True\n\nThe retrieved documentation includes a...,True
3,How can I fetch submissions from a specific fo...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'kobotoolbox', 'doc_type': 'adaptor...","[[SearchResult(text=""---\ntitle: Kobo Toolbox\...",True\n\nThe documentation snippets show how to...,True
4,I want to write data from a workflow to a Goog...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'google_sheets', 'doc_type': 'adapt...",[[SearchResult(text='ng data from CRM systems ...,False\n\nWhile the documentation snippets cont...,False
...,...,...,...,...,...,...,...,...,...,...,...,...
105,"In PostgreSQL, how do I write a query to updat...",general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,False\n\nThe retrieved documentation snippets ...,False
106,What's the best way to handle NULL values in a...,general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,False\n\nWhile the documentation contains info...,False
107,How do I write a SQL query to pivot rows into ...,general_coding_questions,False,False,False,False,False,False,"[{'query': 'sql pivot dynamic columns', 'doc_t...",[[SearchResult(text=')### Sample Job Expressio...,False\n\nThe retrieved documentation snippets ...,False
108,"In PostgreSQL, how can I use the JSONB data ty...",general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,False\n\nThe retrieved documentation snippets ...,False


In [62]:
# see how many are returned at a threshold
threshold = 0.79
results = []

for user_question, queries, eval in zip(df.question.to_list(), df.search_query_answer_v1.to_list(), df.evaluation.to_list()):
    queries = queries.replace("None", "null")
    queries = queries.replace("'", '"')
    queries = json.loads(queries)
    docs = []
    for q in queries:
        result = docsite_search.search(q.get("query"), top_k=30, threshold=threshold, docs_type=q.get("doc_type"))
        docs.extend(result)
    results.append([user_question, queries, docs, len(docs), len(queries), threshold, eval])

thre_df = pd.DataFrame(results, columns=["question", "queries", "docs_retrieved", "num_docs", "num_queries", "threshold", "orig_eval"])
thre_df.to_csv(f"thre_eval_{threshold}.csv", index=False)
thre_df.num_docs.mean()

INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 0 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 0 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 7 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retrieved: 30
INFO:DocsiteSearch:Filtered to 12 results
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 

4.109090909090909

6.94 mean at 0.78
4 mean at 0.79
2.218 mean at 0.8
0.736 mean at 0.82

In [68]:
thre_df = pd.read_csv("thre_eval_0.78.csv")

In [70]:
print("max docs retrieved: ", thre_df.num_docs.max())
thre_df_f = thre_df[thre_df["orig_eval"].str.startswith("False")]
thre_df_t = thre_df[thre_df["orig_eval"].str.startswith("True")]
print("avg docs per useful=FALSE", thre_df_f.num_docs.mean())
print("avg docs per useful=TRUE", thre_df_t.num_docs.mean())

max docs retrieved:  34
avg docs per useful=FALSE 5.5813953488372094
avg docs per useful=TRUE 7.82089552238806


orig_eval shows if we origianlly retrieved useful docs at a limit of n=3 (usu max total 6)
hard to set a threshold to capture those but filter others further

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
pc_db = PineconeVectorStore(index_name="docsite", namespace="docsite-20250225", embedding=OpenAIEmbeddings())

In [ ]:
query = "what's asana"

In [14]:
pc_db.similarity_search(query=query, k=10)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(id='56cc39d7-0f22-4f6c-85ac-882f6f28c219', metadata={'doc_title': 'asana', 'docs_type': 'adaptor_docs'}, page_content='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out th

In [ ]:
result = pc_db.similarity_search_with_score(
            query=query,
            k=10,
            # score_threshold=0.5
            # search_kwargs={"score_threshold": 0.5}
        )
result

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/Users/hanna/opt/anaconda3/envs/openfn/lib/python3.11/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[(Document(id='56cc39d7-0f22-4f6c-85ac-882f6f28c219', metadata={'doc_title': 'asana', 'docs_type': 'adaptor_docs'}, page_content='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out t

In [15]:
results[0].metadata

{'doc_title': 'asana', 'docs_type': 'adaptor_docs'}

In [16]:
results = docsite_search.search("what's asana", top_k=10, docs_type='adaptor_docs', doc_title='asana')
results

INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 3


[SearchResult(text='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out the Asana [devloper documentation](/adaptors/packages/asana-docs) to learn how to set up a webhook to push data

In [17]:
results = docsite_search.search("what's asana", top_k=10, docs_type='adaptor_docs')
results

INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 10


[SearchResult(text='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out the Asana [devloper documentation](/adaptors/packages/asana-docs) to learn how to set up a webhook to push data

In [ ]:


folder = "/Users/hanna/openfn/ai_experiments/apollo/services/tmp/split_sections"
files = ["adaptor_functions_chunks.json", "adaptor_docs_chunks.json", "general_docs_chunks.json"]
doc_dict = {}
for file in files:
    with open(f"{folder}/{file}") as f:
        data = json.load(f)
        doc_dict[file[:-5]] = data
doc_dict.keys()

dict_keys(['adaptor_functions_chunks', 'adaptor_docs_chunks', 'general_docs_chunks'])

In [33]:
for d in doc_dict.keys():
    print(f"{d}")
    print(len(doc_dict[d]))


adaptor_functions_chunks
315
adaptor_docs_chunks
171
general_docs_chunks
596


In [ ]:


doc_types = ["adaptor_functions", "adaptor_docs", "general_docs"]
# doc_types = ["adaptor_functions"]
doc_dict = {}
for d in doc_types:
    doc_list = github_utils.get_docs(docs_type=d)
    doc_dict[d] = doc_list

doc_dict[d][:2]

INFO:GitHubUtils:Fetched 38 URLs from GitHub for https://api.github.com/repos/OpenFn/docs/contents/adaptors
INFO:GitHubUtils:Downloaded and processed 38 files from GitHub
INFO:GitHubUtils:{'name': 'asana.md', 'docs': '---\ntitle: Asana Adaptor\n---\n\n## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks. \n\n## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to

[{'name': 'build-compliant-apps.md',
  'docs': '---\ntitle: Developing connected applications\nsidebar_label: Building compliant APIs\n---\n\nThis section is for you if you are hoping to build or extend an existing\napplication that can connect to OpenFn. We follow modern web-standard JSON API\nguidelines.\n\nFor your application to a be data provider (or "source") for OpenFn\nintegrations, we highly recommend that you create a "notifications service"\n(sometimes called a "webhooks service" or "event-based push API"). This is\npreferable to using a REST api for two reasons: (1) A notifications service will\ngive your clients the ability to set up real-time integrations, and (2) a\nnotifications service is more efficient for both your servers and OpenFn—instead\nof having requests be made and handled every X seconds, your servers and\nOpenFn\'s servers will only work when new data is available.\n\nFor your application to be a consumer (or "destination") for OpenFn, you must\neither have

In [87]:
genral_docs_topics = [d["name"][:-3] for d in doc_dict["general_docs"]]
genral_docs_topics[:5]

['build-compliant-apps',
 'build-with-api',
 'cli-challenges',
 'cli-collections',
 'cli-intro']

In [89]:
adaptor_docs_topics = [d["name"][:-3] for d in doc_dict["adaptor_docs"]]
print(len(adaptor_docs_topics))
adaptor_docs_topics[:5]

38


['asana', 'cht', 'ckan', 'collections', 'commcare']

TODO Test SQLite for searching by column value

In [ ]:
# user_question = "How to inverse columns in javascript"
# user_question = "Given a JSON data, and given a list of columns in an sql table, give me an insert query"
# user_question = """write a job for me that creates new datavaluesets under the "Approved School CHP" organization unit"""
# user_question = """How to get data from asana in my job"""
# user_question = "What's the difference between a Trigger and a Step in a workflow?"
user_question = "“Can I write a step that allows for creation of an JSON file of the transformed records that can be sent to an email or Google Drive”"
# user_question = "Why does this part of the code only POST the first submission in the submissions[] array? I want to POST _each_ submission. each( '$.submissions[*]', post(URL, { body: () => {} ))”"
# user_question = "write a job to fetch for an attribute in the input and map it"

In [54]:
# Needs docs T/F ?

needs_docs_system_prompt = """
You are an assistant for a solutions engineer helping a user write a job for our platform.
Our platform is OpenFn (Open Function Group), the world's leading digital public good for workflow automation.
Your job is to decide whether the user question requires consulting our documentation. If the question is
about general coding advice or other external information, we do not need to consult the documentation. 

Answer nothing but True or False.
"""

needs_docs_user_prompt = """The user question is as follows: "{user_question}" """

def needs_docs(user_question):
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022", # TODO change to cheaper model
        max_tokens=1000,
        temperature=0,
        system=needs_docs_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": needs_docs_user_prompt.format(user_question=user_question)
                    }
                ]
            }
        ]
    )
    return message.content[0].text

In [74]:
# Which search AND how?

# system_prompt = """
# You are an assistant for a solutions engineer helping a user write a job for our platform.
# Our platform is OpenFn (Open Function Group), the world's leading digital public good for workflow automation.
# Your job is to query the documenation based on the user's question.  

# Your answer should detail one or more searches that should be done on the documentation.
# The options for search types are: 
# a) semantic search 
# b) direct retrieval by section (options: "adaptor_documentation", "general_documentation")

# Answer in nothing but code, using one or both of these functions, one or more times, filling in an appropriate input: 

# semantic_search(query)
# database_search(documentation_section)
# """
#  ------------------------------
# system_prompt = """
# You are an assistant for a solutions engineer helping a user write a job for our platform.
# Our platform is OpenFn (Open Function Group), the world's leading digital public good for workflow automation.
# Your job is to query the documenation based on the user's question.  

# Your answer should detail one or more searches that should be done on the documentation.
# The options for search types are: 
# a) semantic search 
# b) direct retrieval by section (options: "adaptor_documentation", "general_documentation")

# You can use one or both of these functions, one or more times: 

# semantic_search(query)
# database_search(documentation_section_query)

# list which ones you would like to use in dictionaries on new lines:
# {"function": "semantic_search", "query": "your_query"}
# {"function":"database_search", "query": "adaptor_documentation"}

# Return nothing but this list of dictionaries with "function" and "query" keys.
# """
#  ------------------------------
system_prompt = """
You are an assistant for a solutions engineer helping a user write a job for our platform.
Our platform is Open Function, a platform for workflow automation.
Your job is to run search queries on the documentation based on the user's question.  

Your answer should detail the searches that should be done on the documentation.
Stick to one search query only per topic. If the user question requires information across
distinct topics (e.g. two different adaptors; CLI and API instructions), then list additional queries.

A search consists of a query string, and an optional filter for doc_type. 
This can be either adaptor_docs or general_docs.
If the filter is adaptor_documentation, then the query should be just the name of the adaptor.

Return a JSON array of search queries:
[
  {"query": "your_query", "doc_type": null},
  {"query": "optional_second_query", "doc_type": "adaptor_docs"}
]

Return NOTHING but this JSON array.
"""
#  ------------------------------

user_prompt = """The user question is as follows: "{user_question}" """

# # fucnution plus query
# response_schemas = [
#     ResponseSchema(name="function", description="The function to be called"),
#     ResponseSchema(name="query", description="The query for the function")
# ]

# parser = StructuredOutputParser.from_response_schemas(response_schemas)

# # query plus filter
# response_schemas = [
#     ResponseSchema(name="query", description="The function to be called"),
#     ResponseSchema(name="doc_type", description="The doc_type filter for the function")
# ]

# parser = StructuredOutputParser.from_response_schemas(response_schemas)

def search_docs(user_question):
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022", # TODO change to cheaper model
        max_tokens=1000,
        temperature=0,
        system=system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt.format(user_question=user_question)
                    }
                ]
            }
        ]
    )
    text = message.content[0].text
    # print("answer: ", text)
    # answer_parsed = [parser.parse(a) for a in text.split("\n")]
    answer_parsed = json.loads(text)

    return answer_parsed

In [ ]:
# answer = search_docs(user_question)
# print(answer)
# answer_parsed = [parser.parse(a) for a in answer.split("\n")]
# answer_parsed

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{"function": "semantic_search", "query": "difference between trigger and step in workflow"}
{"function": "database_search", "query": "general_documentation"}


[{'function': 'semantic_search',
  'query': 'difference between trigger and step in workflow'},
 {'function': 'database_search', 'query': 'general_documentation'}]

In [ ]:
user_question

In [35]:
answer = search_docs(user_question)
print(answer)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
answer:  [
  {"query": "dhis2", "doc_type": "adaptor_documentation"},
  {"query": "api token authentication", "doc_type": null}
]
[{'query': 'dhis2', 'doc_type': 'adaptor_documentation'}, {'query': 'api token authentication', 'doc_type': None}]


In [ ]:
# Get adaptor doc by title or rever to semantic search?
def get_adaptor_documentation(query):
   if query.lower() in adaptor_docs_topics:
        target_section = query.lower()
   else:
    #  search docsite-topics
    #  target_section = search_result
    # result = fetch_local(doc=target_section)
    
    return result

search_results = []
for q in answer:
    query_text = q.get("query", "")
    if query_text:
        doc_type = q.get("doc_type", None)
        if doc_type == "adaptor_documentation":
          results = get_adaptor_documentation(query_text)  
        # results = docsite_search.search(query_text, top_k=5, docs_type=doc_type)
        search_results.extend(results)
search_results

INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5


[SearchResult(text='---\ntitle: Asana Adaptor\n---## About Asana\n\n[Asana](https://app.asana.com/) is a web-based project management tool that helps teams organize, plan, collaborate, and execute tasks.## Integration Options\n\nAsana supports 2 primary integration options:\n\n1. Rest API: Asana has an available REST API that enable external services like OpenFn to pull data from Asana, or push data from external apps to Asana. This option is suited for scheduled, bulk syncs or workflows that must update data in Asana with external information. See [functions](/adaptors/packages/asana-docs) for more on how to use this adaptor to work with the API.\n\n2. Webhook: Asana also has a [Webhook or Data Forwarding](https://developers.asana.com/docs/webhooks-guide) to push data from Asana to external systems. This option is suited for real-time, event-based data integration. Check out the Asana [devloper documentation](/adaptors/packages/asana-docs) to learn how to set up a webhook to push data

In [ ]:
# needs docs?
# parallel a: should we fetch one of more of the following docs sections?
# parallel b: llm: run semantic search
# dedup & combine

In [133]:
# Pick from topics

system_prompt = """
You are an assistant for a solutions engineer helping a user write a job for our platform.
Our platform is OpenFn (Open Function Group), the world's leading digital public good for workflow automation.
Your job is to query the documentation based on the user's question.  

Your answer should pick titles to retreive from the list below. Where possible, stick to only one title.

Titles to pick from:
{document_titles}

Answer in nothing but the list of titles on new lines.
"""

#  ------------------------------

user_prompt = """The user question is as follows: "{user_question}" """

def select_title(user_question, document_titles):
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022", # TODO change to cheaper model
        max_tokens=1000,
        temperature=0,
        system=system_prompt.format(document_titles=document_titles),
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_prompt.format(user_question=user_question)
                    }
                ]
            }
        ]
    )
    text = message.content[0].text

    return text

In [134]:
select_title(user_question, adaptor_docs_topics)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


'http\nsftp'

In [ ]:
# Compile test set
test_questions = []
test_types = ["platform_questions", "specific_coding_questions", "workflow_overview_questions"]
for test_type, test in zip(test_types, [platform_questions, specific_coding_questions, workflow_overview_questions]):
    questions = [t for t in test.split("\n")[:10] if t] #TODO limiting size now
    for q in questions:
        test_questions.append({"question_type":test_type, "question":q})

In [18]:
import os
from typing import List, Optional, Dict, Any
from dataclasses import dataclass
from anthropic import (
    Anthropic,
    APIConnectionError,
    BadRequestError,
    AuthenticationError,
    PermissionDeniedError,
    NotFoundError,
    UnprocessableEntityError,
    RateLimitError,
    InternalServerError,
)
from util import ApolloError, create_logger
# from .prompt import build_prompt
from job_chat.prompt import build_prompt

logger = create_logger("job_chat")


@dataclass
class Payload:
    """
    Data class for validating and storing input parameters.
    Required fields will raise TypeError if not provided.
    """

    content: str
    context: Optional[str] = None
    api_key: Optional[str] = None

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "Payload":
        """
        Create a Payload instance from a dictionary, validating required fields.
        """
        if "content" not in data:
            raise ValueError("'content' is required")

        return cls(content=data["content"], context=data.get("context"), api_key=data.get("api_key"))


@dataclass
class ChatConfig:
    model: str = "claude-3-5-sonnet-20240620"
    max_tokens: int = 1024
    api_key: Optional[str] = None


@dataclass
class ChatResponse:
    content: str
    history: List[Dict[str, str]]
    usage: Dict[str, Any]


class AnthropicClient:
    def __init__(self, config: Optional[ChatConfig] = None):
        self.config = config or ChatConfig()
        self.api_key = self.config.api_key or os.getenv("ANTHROPIC_API_KEY")
        if not self.api_key:
            raise ValueError("API key must be provided")
        self.client = Anthropic(api_key=self.api_key)

    def generate(
        self, content: str, history: Optional[List[Dict[str, str]]] = None, context: Optional[str] = None
    ) -> ChatResponse:
        """
        Generate a response using the Claude API with improved error handling and response processing.
        """
        history = history.copy() if history else []

        system_message, prompt = build_prompt(content, history, context)

        message = self.client.beta.prompt_caching.messages.create(
            max_tokens=self.config.max_tokens, messages=prompt, model=self.config.model, system=system_message
        )

        if hasattr(message, "usage"):
            if message.usage.cache_creation_input_tokens:
                logger.info(f"Cache write: {message.usage.cache_creation_input_tokens} tokens")
            if message.usage.cache_read_input_tokens:
                logger.info(f"Cache read: {message.usage.cache_read_input_tokens} tokens")

        response_parts = []
        for content_block in message.content:
            if content_block.type == "text":
                response_parts.append(content_block.text)
            else:
                logger.warning(f"Unhandled content type: {content_block.type}")

        response = "\n\n".join(response_parts)

        updated_history = history + [
            {"role": "user", "content": content},
            {"role": "assistant", "content": response},
        ]

        return ChatResponse(
            content=response,
            history=updated_history,
            usage=message.usage.model_dump() if hasattr(message, "usage") else {},
        )


def main(data_dict: dict) -> dict:
    """
    Main entry point with improved error handling and input validation.
    """
    try:
        data = Payload.from_dict(data_dict)

        config = ChatConfig(api_key=data.api_key) if data.api_key else None
        client = AnthropicClient(config)

        result = client.generate(content=data.content, history=data_dict.get("history", []), context=data.context)

        return {"response": result.content, "history": result.history, "usage": result.usage}

    except ValueError as e:
        raise ApolloError(400, str(e), type="BAD_REQUEST")

    except APIConnectionError as e:
        raise ApolloError(
            503,
            "Unable to reach the Anthropic AI Service",
            type="CONNECTION_ERROR",
            details={"cause": str(e.__cause__)},
        )
    except AuthenticationError as e:
        raise ApolloError(401, "Authentication failed", type="AUTH_ERROR")
    except RateLimitError as e:
        raise ApolloError(
            429, "Rate limit exceeded, please try again later", type="RATE_LIMIT", details={"retry_after": 60}
        )
    except BadRequestError as e:
        raise ApolloError(400, str(e), type="BAD_REQUEST")
    except PermissionDeniedError as e:
        raise ApolloError(403, "Not authorized to perform this action", type="FORBIDDEN")
    except NotFoundError as e:
        raise ApolloError(404, "Resource not found", type="NOT_FOUND")
    except UnprocessableEntityError as e:
        raise ApolloError(422, str(e), type="INVALID_REQUEST")
    except InternalServerError as e:
        raise ApolloError(500, "The Anthropic AI Service encountered an error", type="PROVIDER_ERROR")
    except Exception as e:
        logger.error(f"Unexpected error during chat generation: {str(e)}")
        raise ApolloError(500, str(e))


In [103]:
from job_chat.prompt import build_prompt, Context

apollo_prompt = build_prompt(content="What's asana", history=None, context=Context(adaptor="@openfn/language-http@2.0.0"))
apollo_prompt

TypeError: string indices must be integers, not 'str'

In [98]:
from util import create_logger, apollo

# logger = create_logger("job_chat.prompt")

system_role = """
You are a software engineer helping a non-expert user write a job for our platform.
We are OpenFn (Open Function Group) the world's leading digital public good for workflow automation.

Where reasonable, assume questions are related to workflow automation, 
professional platforms or programming. You may provide general information around these topics, 
e.g. general programming assistance unrelated to job writing.
If a question is entirely irrelevant, do not answer it.

You MUST keep your responses concise. Do not explain your answers unless
the user explicitly asks you to. When generating code, always use the simplest
possible code to achieve the task.

Do not thank the user or be obsequious. Address the user directly.

You are embedded in our app for building workflows. Our app will provide the
history of each chat session to you. Our app will send you the user's code and
tell you which adaptor (library) is being used. We will not send you the user's 
input data, output data, or logs, because they might contain sensitive information. 
Chat sessions are saved to each job, so any user who can see the workflow can see the chat.

Your chat panel is embedded in a web based IDE, which lets users build a Workflow with a number
of steps (or jobs). There is a code editor next to you, which users can copy and paste code into.
Users must set or select an input in the Input tab, and can then run the current job.

Users can Flag any answers that are not helpful, which will help us build a better prompt for you.
"""

job_writing_summary = """
<credential management>
When writing jobs, users will use their own credentials to access different
backend systems. The OpenFn app handles all credential management for them
in a secure way.

For more help direct them to https://docs.openfn.org/documentation/build/credentials

Users must never add credentials into job code directly. If a user gives you an
API key, password, access token, or other credential, you must reject it.
</credential management>
<job writing guide>
An OpenFn Job is written in a DSL which is very similar to Javascript.

Job code does not use import statements or async/await.

Job code must only contain function calls at the top level.

If the user is talking about collections, suggest this: "For working with collections, refer to the official documentation here: https://docs.openfn.org/adaptors/packages/collections-docs.".
Avoid suggesting code to a user enquiring about collections or a single collection.

Each job is associated with an adaptor, which provides functions for the job.
All jobs have the fn() and each() function, which are very important.

DO NOT use the `alterState()` function. Use `fn()` instead.

The adaptor API may be attached.

The functions provided by an adaptor are called Operations.
Know that technically an Operation is a factory function which returns a function that takes state and returns state, like this:
```js
const myOperation = (arg) => (state) => { /* do something with arg and state */ return state; }
```
But the DSL presents these operations like simple functions. Users don't know it's a factory, they think it's a regular function.
<examples>
<example>
Here's how we issue a GET request with the http adaptor:
```
get('/patients');
```
The first argument to get is the path to request from (the configuration will tell
the adaptor what base url to use). In this case we're passing a static string,
but we can also pass a value from state:
```
get(state => state.endpoint);
```
</example>
<example>
Example job code with the HTTP adaptor:
```
get('/patients');
fn(state => {
  const patients = state.data.map(p => {
    return { ...p, enrolled: true }
  });

  return { ...state, data: { patients } };
})
post('/patients', dataValue('patients'));
</example>
<example>
```
Example job code with the Salesforce adaptor:
```
each(
  '$.form.participants[*]',
  upsert('Person__c', 'Participant_PID__c', state => ({
    Participant_PID__c: state.pid,
    First_Name__c: state.participant_first_name,
    Surname__c: state.participant_surname,
  }))
);
```
</example>
<example>
Example job code with the ODK adaptor:
```
create(
  'ODK_Submission__c',
  fields(
    field('Site_School_ID_Number__c', dataValue('school')),
    field('Date_Completed__c', dataValue('date')),
    field('comments__c', dataValue('comments')),
    field('ODK_Key__c', dataValue('*meta-instance-id*'))
  )
);
```
</example>
<examples>
</job writing guide>
<workflow guide>
A job is just one step in a workflow (or pipeline). Workflows are used
to automate processes and migrate data from system to system.

In OpenFn, each step works with a single backend system, or adaptor. Data is shared
between steps through the state object.

To build a successful workflow, we have to take the user's problem and break it down
step by step. Focus on one bit at a time. For example, when uploading from commcare to salesforce, we have to:
1. Download our data from commcare in one step
2. Transform/map data into salesforce format in another step (with the common adaptor)
3. Upload the transformed data into salesforce in the final step
</workflow guide>
"""


class Context:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def has(self, key):
        return hasattr(self, key) and getattr(self, key) is not None


def generate_system_message(context_dict):
    context = context_dict if isinstance(context_dict, Context) else Context(**context_dict)

    message = [system_role]
    message.append(f"<job_writing_guide>{job_writing_summary}</job_writing_guide>")
    message.append({"type": "text", "text": ".", "cache_control": {"type": "ephemeral"}})

    if context.has("adaptor"):
        adaptor_string = (
            f"<adaptor>The user is using the OpenFn {context.adaptor} adaptor. Use functions provided by its API."
        )

        adaptor_docs = apollo("describe_adaptor", {"adaptor": context.adaptor})

        for doc in adaptor_docs:
            adaptor_string += f"Typescript definitions for doc {doc}"
            adaptor_string += adaptor_docs[doc]["description"]
        adaptor_string += "</adaptor>"

        message.append(adaptor_string)
    else:
        message.append("The user is using an OpenFn Adaptor to write the job.")

    message.append({"type": "text", "text": ".", "cache_control": {"type": "ephemeral"}})

    if context.has("expression"):
        message.append(f"<user_code>{context.expression}</user_code>")

    if context.has("input"):
        message.append(f"<input>The user's input data is :\n\n```{context.input}```</input>")

    if context.has("output"):
        message.append(f"<output>The user's last output data was :\n\n```{context.output}```</output>")

    if context.has("log"):
        message.append(f"<log>The user's last log output was :\n\n```{context.log}```</log>")

    return list(map(lambda text: text if isinstance(text, dict) else {"type": "text", "text": text}, message))


def build_prompt(content, history, context):
    system_message = generate_system_message(context)

    prompt = []
    prompt.extend(history)
    prompt.append({"role": "user", "content": content})

    return (system_message, prompt)


In [105]:
context = Context(adaptor="@openfn/language-googlesheets@3.0.5")
adaptor_string = (
    f"<adaptor>The user is using the OpenFn {context.adaptor} adaptor. Use functions provided by its API."
)

adaptor_docs = apollo("describe_adaptor", {"adaptor": context.adaptor})

for doc in adaptor_docs:
    adaptor_string += f"Typescript definitions for doc {doc}"
    adaptor_string += adaptor_docs[doc]["description"]
adaptor_string += "</adaptor>"
adaptor_string

'<adaptor>The user is using the OpenFn @openfn/language-googlesheets@3.0.5 adaptor. Use functions provided by its API.Typescript definitions for doc @openfn/language-common/**\n * Execute a sequence of operations.\n * Main outer API for executing expressions.\n * @public\n * @example\n *  execute(\n *    create(\'foo\'),\n *    delete(\'bar\')\n *  )\n * @private\n * @param {Operations} operations - Operations to be performed.\n * @returns {Promise}\n */\nexport function execute(...operations: Operations): Promise<any>;\n/**\n * alias for "fn()"\n * @function\n * @param {Function} func is the function\n * @returns {Operation}\n */\nexport function alterState(func: Function): Operation;\n/**\n * Creates a custom step (or operation) for more flexible job writing.\n * @public\n * @function\n * @example\n * fn(state => {\n *   // do some things to state\n *   return state;\n * });\n * @param {Function} func is the function\n * @returns {Operation}\n */\nexport function fn(func: Function): 

In [24]:
platform_questions = [
    "How do I create a new workflow in OpenFn?",
    "What's the difference between a Trigger and a Step in a workflow?",
    "Can I merge branches or skip steps in my workflow?",
    "How do I troubleshoot errors in my workflow?",
    "What do the different status codes mean for a workflow run?",
    "How can I optimize my workflow if it's timing out?",
    "What should I include in my run logs to make troubleshooting easier?",
    "How do I handle mapping errors between source and destination systems?",
    "What does the 'DUPLICATE_VALUE' error mean and how do I resolve it?",
    "Can I use OpenFn to edit source data before retrying a failed attempt?",
    "How do I set up a request and approval process in my workflow?",
    "What's the best way to handle data transformation in OpenFn jobs?",
    "Can OpenFn auto-generate jobs based on Kobo forms?",
    "How do I access the OpenFn platform to start creating workflows?",
    "What programming knowledge do I need to write complex jobs in OpenFn?",
    "How can I design error messages to be more user-friendly and actionable?",
    "Is it possible to auto-correct minor issues in my workflow?",
    "How do I manually run a workflow for testing and troubleshooting?",
    "Does OpenFn support two-way syncing between applications?",
    "Can I set up time-based triggers for my workflows, like running a job every Friday at 11 PM?"
]

In [23]:
adaptor_questions = [
    "Is there a way to handle pagination when pulling large datasets from an API? I'm running into memory issues using a standard approach.",
    "When authenticating with API tokens instead of a username/password, what's the best way to configure authentication in an integration with DHIS2?",
    "I need to update a Contact record in Salesforce. Can you provide a code snippet to update the FirstName and LastName based on data from a previous step?",
    "How can I fetch submissions from a specific form within a certain date range in KoboToolbox? Can you provide an example?",
    "I want to write data from a workflow to a Google Sheet. How do I specify the sheet ID and range? What permissions are needed for the service account?",
    "Is it possible to send an SMS message with variables populated from data in my workflow? What's the correct syntax for handling this dynamically?",
    "When querying a PostgreSQL database within an OpenFn job, how do I construct the SQL query? Can I use parameters to prevent SQL injection?",
    "How does OpenMRS handle searching for patients based on multiple identifiers in an integration scenario?",
    "I'm testing my CommCare integration. How can I verify that a case property has been updated correctly?",
    "When pushing data to Aggregate in an ODK workflow, how can I confirm successful submission within the OpenFn logs?",
    "My workflow involves transferring files via SFTP. What are the common error codes, and how should I handle them to ensure resilience?",
    "I need to connect to an API that doesn’t have a specific adaptor available. How do I send a PATCH request with a custom header?",
    "How can I trigger a RapidPro flow while passing in custom variables?",
    "I want to load data into BigQuery. What's the recommended approach for handling large datasets and schema transformations?",
    "I've upgraded our Dynamics 365 instance. Do I need to update anything in OpenFn to ensure compatibility? How do I check the version and update if necessary?",
    "I'm running into authentication failures when integrating with Mailchimp",
    "What are the security considerations when storing sensitive data in Azure Storage? How do I manage access keys securely?",
    "What are the recommended best practices for ensuring data quality and consistency when syncing data with Primero?",
    "How does OpenHIM work in conjunction with other integration tools in a complex workflow? Can you provide an example?",
    "If a record fails to upload to OpenMRS due to a validation error, how can we configure the workflow to retry the operation a certain number of times before moving on?"
]



In [22]:
specific_questions = [
    # "How do I configure the HTTP adapter to handle OAuth 2.0 authentication for connecting to a REST API? Can you provide an example of setting up the headers and body for a POST request using the HTTP adapter?",  
    # "What's the best way to handle upserts in Salesforce using the Salesforce adapter, and how do I manage external IDs effectively?",  
    # "How can I use the KoboToolbox adapter to trigger a workflow every time a new submission is received on a specific form?",  
    # "How do I configure the CommCare adapter to pull data based on a specific date range or case property?",  
    # "Can you walk me through an example of using the DHIS2 adapter to push aggregated data to a specific data element and period?",  
    # "How do I use the each() function within a job to process an array of data and create multiple records in the destination system?",  
    "How can I implement custom error handling logic to retry failed operations with exponential backoff?",  
    "What are some best practices for using the transform function to reshape data before sending it to the destination system?",  
    "Can I set up a workflow to trigger based on changes in a database using a webhook? How?",  
    "How can I securely manage API keys and other sensitive information using environment variables in OpenFn?",  
    "Does OpenFn offer any built-in support for encrypting sensitive data in transit or at rest?",  
    "How can I customize the logging level and output format for my workflows?",  
    "What's the recommended workflow for using Git to manage and deploy changes to my OpenFn workflows?",  
    "How can I write unit tests for my OpenFn jobs to ensure they are working correctly?",  
    "How can I set up alerts to notify me when a workflow fails or exceeds a certain execution time?",  
    "What's the most efficient way to process large datasets (e.g., millions of records) using OpenFn?",  
    "Can I import and use custom JavaScript libraries in my OpenFn jobs? How?",  
    "How do I handle nested JSON structures when mapping data between systems?",  
    "How can I implement rate limiting in my workflow to avoid exceeding the API limits of the destination system?",  
    "How can I use the OpenFn CLI to deploy changes to my workflows and manage my OpenFn instance?"  
]


In [21]:
workflow_overview_questions = [
    "How do I configure OpenFn to listen for incoming SMS messages and trigger a workflow based on the message content?",
    "Can you help me set up a job that automatically exports data from CommCare and imports it into a DHIS2 instance?",
    "I need to create a workflow that synchronizes patient data between OpenMRS and a national health registry. What's the best approach?",
    "How can I use OpenFn to transform data from a CSV file into FHIR resources and load them into a FHIR server?",
    "I want to set up automated alerts based on data thresholds in my system. How can I achieve this with OpenFn?",
    "We're using Salesforce and need to integrate it with our accounting system. Can you help us build a workflow to synchronize invoice data?",
    "How can I use the OpenFn CLI to deploy and manage my workflows across multiple environments (dev, staging, production)?",
    "I need to implement a data quality check within my workflow. How can I use JavaScript within OpenFn to validate data and handle errors?",
    "How can I set up OpenFn to automatically retry failed tasks with exponential backoff?",
    "I'm working with a complex data model in ActivityInfo. Can you help me write a job to efficiently extract and transform this data?",
    "How do I configure OpenFn to use OAuth 2.0 for authentication with a REST API?",
    "What are the best practices for handling Personally Identifiable Information (PII) within OpenFn workflows?",
    "How can I use OpenFn to create a scheduled task that runs daily and generates reports based on data from multiple systems?",
    "I need to integrate data from a legacy system that only supports XML. How can I transform XML data into JSON for use in my workflows?",
    "Can you help me set up monitoring and logging for my OpenFn workflows?",
    "How do I write a job to take data from a Google Sheet and use that data to create a new user in Keycloak?",
    "How can I implement a workflow that handles data conflicts between two systems, using a defined conflict resolution strategy?",
    "I need to trigger a workflow based on changes in a cloud storage bucket (e.g., AWS S3, Google Cloud Storage). How can I set this up?",
    "Can you provide guidance on optimizing the performance of my OpenFn workflows, especially when dealing with large datasets?",
    "How do I use OpenFn to implement a bulk data import process, breaking the data into smaller batches to avoid API rate limits?"
]


In [20]:
specific_coding_questions = [
    "How do I use the each() function to iterate over an array of patients and create individual records in the destination system?",
    "What's the correct syntax for using the $ operator to access nested data within the state object?",
    # "How can I implement error handling using try/catch blocks within a job?",
    "What's the proper way to use the dataValue() function to extract specific fields from the incoming data?",
    "How do I use the get() function with the HTTP adapter to fetch data from an external API?",
    # "Can you show me how to use string interpolation to dynamically construct API endpoints in a get() or post() operation?",
    # "What's the syntax for using the map() function to transform an array of objects before sending them to the destination system?",
    "How do I use the upsert() function with the Salesforce adapter to update or insert records based on an external ID?",
    "What's the correct way to use the fn() function to execute custom JavaScript code within a job?",
    "How can I use the post() function with the HTTP adapter to send transformed data to another system?",
    "What's the syntax for using the create() function with the DHIS2 adapter to create new data elements?",
    "How do I use the alterState() function to modify the state object between operations?",
    "Can you show me how to use the field() function to map data from one schema to another?",
    "What's the proper way to use the combine() function to merge data from multiple sources?",
    "How do I use the sourceValue() function to reference values from the trigger source in my job?",
    "What's the syntax for using the relationship() function to establish relationships between objects in Salesforce?",
    # "How can I use the then() method to chain operations and handle asynchronous tasks?",
    "What's the correct way to use the execute() function to run a custom SQL query in a database adapter?",
    "How do I use the scrub() function to clean and standardize data before sending it to the destination system?",
    "Can you show me how to use the chunk() function to process large datasets in smaller batches to avoid memory issues?"
]


In [19]:
general_coding_questions = [
    "How do I use a JavaScript Promise to handle asynchronous API calls in a workflow?",
    "What's the best way to parse a complex JSON response from an external API in JavaScript?",
    "How can I use JavaScript's reduce() function to aggregate data from multiple API responses?",
    "Is there a way to implement retry logic for failed API requests using JavaScript?",
    "How do I use JavaScript's destructuring assignment to extract specific fields from an API response?",
    "What's the most efficient way to filter an array of objects based on multiple criteria in JavaScript?",
    "How can I use JavaScript's map() function to transform data structures?",
    "Is it possible to use JavaScript's async/await syntax for better readability of asynchronous code?",
    "How do I implement error handling for database operations in JavaScript?",
    "What's the best approach to dynamically construct a SQL query string in JavaScript based on input parameters?",
    "How can I use JavaScript's Date object to format dates according to a specific timezone?",
    "Is there a way to use JavaScript's Set object to remove duplicates from an array of objects?",
    "How do I use JavaScript's fetch API to make HTTP requests to external services?",
    "What's the most efficient way to merge data from multiple API responses using JavaScript objects?",
    "How can I use JavaScript's regular expressions to validate and clean input data?",
    "In PostgreSQL, how do I write a query to update multiple rows with different values in a single statement?",
    "What's the best way to handle NULL values in a PostgreSQL query when joining tables?",
    "How do I write a SQL query to pivot rows into columns dynamically based on the data received?",
    "In PostgreSQL, how can I use the JSONB data type to store and query complex JSON structures?",
    "What's the most efficient way to bulk insert a large number of records into a PostgreSQL database from a JavaScript array?"
]

In [18]:
import pandas as pd

def create_test_set(adaptor_questions, platform_questions, specific_questions, 
                            workflow_overview_questions, specific_coding_questions, general_coding_questions):    
        all_queries = []
        all_types = []
        
        # Add each category to the lists
        for questions, q_type in [
            (adaptor_questions, 'adaptor_questions'),
            (platform_questions, 'platform_questions'),
            (specific_questions, 'specific_questions'),
            (workflow_overview_questions, 'workflow_overview_questions'),
            (specific_coding_questions, 'specific_coding_questions'),
            (general_coding_questions, 'general_coding_questions')
        ]:
            all_queries.extend(questions)
            all_types.extend([q_type] * len(questions))
        
        # Create the base dataframe
        df = pd.DataFrame({
            'question': all_queries,
            'question_type': all_types
        })
        
        # Add the label columns according to the specified rules
        df['consult_doc_label'] = ~df['question_type'].isin(['general_coding_questions'])
        df['consult_adaptor_docs_label'] = df['question_type'].isin(['adaptor_questions', 'specific_coding_questions'])
        df['consult_general_docs_label'] = df['question_type'].isin(['platform_questions', 
                                                            'specific_questions', 
                                                            'workflow_overview_questions', 'specific_coding_questions'])
        
        return df

In [25]:
test_df = create_test_set(adaptor_questions, platform_questions, specific_questions, 
                            workflow_overview_questions, specific_coding_questions, general_coding_questions)
test_df

,question,question_type,consult_doc_label,consult_adaptor_docs_label,consult_general_docs_label
0,Is there a way to handle pagination when pulli...,adaptor_questions,True,True,False
1,When authenticating with API tokens instead of...,adaptor_questions,True,True,False
2,I need to update a Contact record in Salesforc...,adaptor_questions,True,True,False
3,How can I fetch submissions from a specific fo...,adaptor_questions,True,True,False
4,I want to write data from a workflow to a Goog...,adaptor_questions,True,True,False
...,...,...,...,...,...
105,"In PostgreSQL, how do I write a query to updat...",general_coding_questions,False,False,False
106,What's the best way to handle NULL values in a...,general_coding_questions,False,False,False
107,How do I write a SQL query to pivot rows into ...,general_coding_questions,False,False,False
108,"In PostgreSQL, how can I use the JSONB data ty...",general_coding_questions,False,False,False


In [40]:
needs_docs_answers

[None, None, None, None, None]

In [46]:
needs_docs(user_question)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
True


In [51]:
needs_docs(user_question)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
True


In [62]:
user_question

'Can you show me how to use the chunk() function to process large datasets in smaller batches to avoid memory issues?'

In [65]:
search_docs(user_question)

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


[{'query': 'chunk function batch processing', 'doc_type': None}]

In [53]:
test_df.question.to_list()[:5]

["Is there a way to handle pagination when pulling large datasets from an API? I'm running into memory issues using a standard approach.",
 "When authenticating with API tokens instead of a username/password, what's the best way to configure authentication in an integration with DHIS2?",
 'I need to update a Contact record in Salesforce. Can you provide a code snippet to update the FirstName and LastName based on data from a previous step?',
 'How can I fetch submissions from a specific form within a certain date range in KoboToolbox? Can you provide an example?',
 'I want to write data from a workflow to a Google Sheet. How do I specify the sheet ID and range? What permissions are needed for the service account?']

In [73]:

docsite_search.search(q.get("query"), top_k=7, docs_type="adaptor_docs")

INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 7


[SearchResult(text="If you are a super user configuring a new Google OAuth\nclient for your OpenFn deployment, then please refer to Google's documentation\nto\n[configure an OAuth Client ID](https://developers.google.com/identity/protocols/oauth2/javascript-implicit-flow),\nas well as see the standard[openid-configuration](https://accounts.google.com/.well-known/openid-configuration)\nfor how to complete the OAuth Client Setup form on OpenFn.\n\n:::#### Permissions (Scopes)\n\nPermissions and access in an OAuth instance are defined by scopes which are\nnamed differently by providers based on their functions within their platform.\n\nFor GoogleSheets, your super user will likely need to add the following scopes\nto the Oauth Client Setup configured for Google in your OpenFn deployment. That\nsaid, please refer to\n[Google's documentation on Oauth scopes](https://developers.google.com/identity/protocols/oauth2/scopes)\nfor the latest information.\n\n- `openid`\n- `email`\n- `profile`\n- 

In [ ]:
# Is some relevant T/F?

evaluate_retrieved_docs_system_prompt = """
You are an assistant for evaluating the relevance of code documentation snippets retrieved for a solutions engineer working for a workflow automation platform.

You will receive a user question, and the retrieved documentation snippets. Answer whether any of the snippets contain information
that will help the solutions engineer address the user question.

Answer with True or False, followed by a short 2-3 sentence explanation on a new line.

"""

evaluate_retrieved_docs_user_prompt = """
The user question: "{user_question}" 

The retrieved docs: {retrieved_docs}
"""

def evaluate_retrieved_docs(user_question, retrieved_docs):
    message = client.messages.create(
        model="claude-3-5-sonnet-20241022", # TODO change to cheaper model
        max_tokens=1000,
        temperature=0,
        system=evaluate_retrieved_docs_system_prompt,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": evaluate_retrieved_docs_user_prompt.format(user_question=user_question, retrieved_docs=retrieved_docs)
                    }
                ]
            }
        ]
    )
    return message.content[0].text



In [ ]:
evaluate_retrieved_docs(user_question, retrieved_docs)

In [96]:
# evaluate direct basic rag

out_f = "results_basic_rag_top5.csv"
result_col_name = "search_results_answer_basic_rag_top5"
test_df_results_basic_rag = test_df.copy()
# test_df_results_basic_rag = test_df_results_basic_rag[:2] 

retrieved_docs = []

for user_question in test_df.question.to_list(): #[:2]
    results = docsite_search.search(user_question, top_k=5, docs_type=None)
    retrieved_docs.append(results)
test_df_results_basic_rag[result_col_name] = retrieved_docs

test_df_results_basic_rag.to_csv(out_f, index=False)

evaluations = []
evaluations_bin = []
for q, d in zip(test_df_results_basic_rag.question.to_list(), test_df_results_basic_rag[result_col_name].to_list()):
    evaluation = evaluate_retrieved_docs(q, d)
    evaluations.append(evaluation)
    if evaluation.startswith("True"):
        evaluations_bin.append(True)
    elif evaluation.startswith("False"):
        evaluations_bin.append(False)
    else:
        evaluations_bin.append(None)
test_df_results_basic_rag["evaluation"] = evaluations
test_df_results_basic_rag["evaluations_bin"] = evaluations_bin
print(test_df_results_basic_rag['evaluations_bin'].eq(True).mean())# avg useful 
print(test_df_results_basic_rag[test_df_results_basic_rag["question_type"]!="general_coding_questions"]['evaluations_bin'].eq(True).mean()) # avg useful but ignore genral coding
test_df_results_basic_rag.to_csv(out_f, index=False)
test_df_results_basic_rag


INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/

,question,question_type,consult_doc_label,consult_adaptor_docs_label,consult_general_docs_label,search_results_answer_basic_rag_top5,evaluation,evaluations_bin
0,Is there a way to handle pagination when pulli...,adaptor_questions,True,True,False,"[SearchResult(text=""Best practice is to includ...",True\n\nThe documentation shows multiple appro...,True
1,When authenticating with API tokens instead of...,adaptor_questions,True,True,False,[SearchResult(text='What data formats are ...,False\n\nThe retrieved documentation snippets ...,False
2,I need to update a Contact record in Salesforc...,adaptor_questions,True,True,False,[SearchResult(text='> The source code for comb...,True\n\nThe documentation contains a relevant ...,True
3,How can I fetch submissions from a specific fo...,adaptor_questions,True,True,False,"[SearchResult(text=""Check out:\n\n- Kobo API d...",True\n\nThe documentation snippets show how to...,True
4,I want to write data from a workflow to a Goog...,adaptor_questions,True,True,False,"[SearchResult(text=""---\nsidebar_label: HTTP t...",False\n\nThe provided documentation snippets d...,False
...,...,...,...,...,...,...,...,...
105,"In PostgreSQL, how do I write a query to updat...",general_coding_questions,False,False,False,"[SearchResult(text=""[add_credential_postgres](...",False\n\nThe retrieved documentation snippets ...,False
106,What's the best way to handle NULL values in a...,general_coding_questions,False,False,False,[SearchResult(text='---\ntitle: PostgreSQL\n--...,False\n\nThe retrieved documentation snippets ...,False
107,How do I write a SQL query to pivot rows into ...,general_coding_questions,False,False,False,"[SearchResult(text=""Then, when building your w...",False\n\nThe provided documentation snippets d...,False
108,"In PostgreSQL, how can I use the JSONB data ty...",general_coding_questions,False,False,False,[SearchResult(text='---\ntitle: PostgreSQL\n--...,False\n\nThe retrieved documentation snippets ...,False


In [95]:
# evaluate LLM query reformulation + search with optional adaptor/general docs filter. + optionally evaluate needs rag step effect (do this by discounting the answers with a FALSE answer in needs rag to get the second result)
out_f = "results_v1_top5.csv"

needs_docs_answers = []
consult_adaptor_docs = []
consult_general_docs = []

search_docs_answers = []
search_results = []

for user_question in test_df.question.to_list(): #:5
    needs_docs_answer = needs_docs(user_question)
    search_docs_answer = search_docs(user_question)

    needs_docs_answers.append(needs_docs_answer)
    search_docs_answers.append(search_docs_answer)

    has_adaptor_docs = False
    has_general_docs = False
    
    # Check each document in the results
    for answer in search_docs_answer:
        doc_type = answer.get("doc_type")
        
        if doc_type == "adaptor_documentation":
            has_adaptor_docs = True
        elif doc_type == "general_documentation":
            has_general_docs = True
            
    consult_adaptor_docs.append(has_adaptor_docs)
    consult_general_docs.append(has_general_docs)

    retrieved_docs = []
    for q in search_docs_answer:
        print(q)
        result = docsite_search.search(q.get("query"), top_k=5, docs_type=q.get("doc_type"))
        retrieved_docs.append(result)
    search_results.append(retrieved_docs)

test_df_results = test_df.copy()
# test_df_results = test_df_results[:5]
test_df_results["consult_doc_answer_v1"] = needs_docs_answers
test_df_results["consult_adaptor_docs_answer_v1"] = consult_adaptor_docs
test_df_results["consult_general_docs_answer_v1"] = consult_general_docs
test_df_results["search_query_answer_v1"] = search_docs_answers
test_df_results["search_results_answer_v1"] = search_results

test_df_results.to_csv(out_f, index=False)

# evaluate
evaluations = []
evaluations_bin = []
for q, d in zip(test_df_results.question.to_list(), test_df_results.search_results_answer_v1.to_list()):
    evaluation = evaluate_retrieved_docs(q, d)
    evaluations.append(evaluation)
    if evaluation.startswith("True"):
        evaluations_bin.append(True)
    elif evaluation.startswith("False"):
        evaluations_bin.append(False)
    else:
        evaluations_bin.append(None)
test_df_results["evaluation"] = evaluations
test_df_results["evaluations_bin"] = evaluations_bin
print(test_df_results['evaluations_bin'].eq(True).mean()) # avg useful
print(test_df_results[test_df_results["question_type"]!="general_coding_questions"]['evaluations_bin'].eq(True).mean()) # avg useful but ignore genral coding
test_df_results.to_csv(out_f, index=False)
test_df_results

INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{'query': 'pagination memory large dataset', 'doc_type': 'general_docs'}
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
{'query': 'batch processing chunks', 'doc_type': 'general_docs'}
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:DocsiteSearch:Similar documents retreived: 5
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
{'query': 'DHIS2', 'doc_type': 'adaptor_docs'}
INFO:DocsiteSearch:Metadata filters built
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.

,question,question_type,consult_doc_label,consult_adaptor_docs_label,consult_general_docs_label,consult_doc_answer_v1,consult_adaptor_docs_answer_v1,consult_general_docs_answer_v1,search_query_answer_v1,search_results_answer_v1,evaluation,evaluations_bin
0,Is there a way to handle pagination when pulli...,adaptor_questions,True,True,False,False,False,False,"[{'query': 'pagination memory large dataset', ...",[[SearchResult(text='This typically won\'t hap...,True\n\nThe documentation contains relevant in...,True
1,When authenticating with API tokens instead of...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'DHIS2', 'doc_type': 'adaptor_docs'...","[[SearchResult(text=""# DHIS2## App Overview\n\...",False\n\nWhile the retrieved documentation con...,False
2,I need to update a Contact record in Salesforc...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'salesforce', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: Salesforce\n-...,True\n\nThe documentation includes a code snip...,True
3,How can I fetch submissions from a specific fo...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'kobotoolbox', 'doc_type': 'adaptor...","[[SearchResult(text=""---\ntitle: Kobo Toolbox\...",True\n\nThe retrieved documentation shows how ...,True
4,I want to write data from a workflow to a Goog...,adaptor_questions,True,True,False,True,False,False,"[{'query': 'google_sheets', 'doc_type': 'adapt...",[[SearchResult(text='ng data from CRM systems ...,False\n\nWhile the documentation snippets cont...,False
...,...,...,...,...,...,...,...,...,...,...,...,...
105,"In PostgreSQL, how do I write a query to updat...",general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,False\n\nThe retrieved documentation snippets ...,False
106,What's the best way to handle NULL values in a...,general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,True\n\nThe retrieved documentation contains r...,True
107,How do I write a SQL query to pivot rows into ...,general_coding_questions,False,False,False,False,False,False,"[{'query': 'sql pivot dynamic columns', 'doc_t...","[[SearchResult(text=""For example, imagine that...",False\n\nThe retrieved documentation snippets ...,False
108,"In PostgreSQL, how can I use the JSONB data ty...",general_coding_questions,False,False,False,False,False,False,"[{'query': 'postgresql', 'doc_type': 'adaptor_...",[[SearchResult(text='---\ntitle: PostgreSQL\n-...,False\n\nThe retrieved documentation snippets ...,False


In [91]:
test_df_results[test_df_results["question_type"]!="general_coding_questions"]['evaluations_bin'].eq(True).mean()

0.6666666666666666